In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os

### Loading the Data from the Data Folder

In [ ]:
labels = {'Hello' : 0, 'How are you': 1, 'Thanks': 2, 'Braai' : 3, 'Pap' : 4}

In [ ]:
action_class = []
sequences = []
classes = ['Hello', 'How are you', 'Thanks', 'Braai', 'Pap']
no_sequences = 400
sequence_length = 30

for action in classes:
    for sequence in range(no_sequences):
        window = []
        for frame in range(sequence_length):
            result = np.load(os.path.join('C:\\Users\\Tyron\\Sign_Language_Detection\\SASL_Sign_Language_Detection\\DATA', action, str(sequence), '{}.npy'.format(frame)))
            window.append(result)
        sequences.append(window)
        if action == 'Hello':
                action_class.append(np.array([1, 0, 0, 0, 0]))
        elif action == 'How are you':
                action_class.append(np.array([0, 1, 0, 0, 0]))
        elif action == 'Thanks':
                action_class.append(np.array([0, 0, 1, 0, 0]))
        elif action == 'Braai':
                action_class.append(np.array([0, 0, 0, 1, 0]))
        elif action == 'Pap':
                action_class.append(np.array([0, 0, 0, 0, 1]))

In [ ]:
X = np.array(sequences)
Y = np.array(action_class)

In [ ]:
X.shape

In [ ]:
Y.shape

### Creating Training and Testing Sets

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train, X_valid, = train_test_split(X_train, test_size=0.1, random_state=42)
Y_train, Y_valid = train_test_split(Y_train, test_size=0.1, random_state=42)

# Creating our LSTM model

In [ ]:
LSTM_Model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=256, return_sequences=True, activation='relu', input_shape=(30, 1662)),
    tf.keras.layers.LSTM(units=128, return_sequences=True, activation='relu'),
    tf.keras.layers.LSTM(units=256, return_sequences=True, activation='relu'),
    tf.keras.layers.LSTM(units=64, return_sequences=False, activation='relu'),
    tf.keras.layers.Dense(units=5 , activation='softmax',)
])

In [ ]:
LSTM_Model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])

### Custom Callback to stop training after a certain threshold value

In [ ]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold, loss):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold
        self.loss = loss

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_categorical_accuracy"]
        acc = logs['categorical_accuracy']
        loss = logs['loss']
        val_loss = logs['val_loss']
        if val_acc >= self.threshold and acc >= self.threshold and loss <= self.loss and val_loss <= self.loss:
            self.model.stop_training = True

### Training our Model

In [ ]:
my_callback = MyThresholdCallback(threshold=0.95, loss=0.1)
LSTM_Model_Hist = LSTM_Model.fit(x=X_train, y=Y_train, validation_data=(X_valid, Y_valid), epochs=1000, callbacks=[my_callback], batch_size=1)

### Saving the Model

In [ ]:
LSTM_Model.save('LSTM_Model_Final_2.h5', overwrite=True)

### Testing Model Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model = tf.keras.models.load_model('C:\\Users\\Tyron\\Sign_Language_Detection\\SASL_Sign_Language_Detection\\LSTM_Model_Final_2.h5')

In [ ]:
predictions = model.predict(X_test)

In [ ]:
y_pred = []
for x in predictions:
    pred_arg = np.argmax(x)
    y_pred.append(pred_arg)

In [ ]:
y_pred = np.array(y_pred)

In [ ]:
y_true = []
for x in range(0, len(Y_test)):
    y_arg = np.argmax(Y_test[x])
    y_true.append(y_arg)

y_true = np.array(y_true)

In [ ]:
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)
print('The accuracy score on the Test data is {}'.format(acc_score))